In [30]:
import numpy as np
import pandas as pd

# DATA IMPORTING

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
training_data = pd.read_csv("/content/drive/MyDrive/Training.csv")
training_data_date = pd.read_csv("/content/drive/MyDrive/Matches IPL 2008-2019.csv")
current_data_date = pd.read_csv("/content/drive/MyDrive/Matches IPL 2020.csv")
submission = pd.read_csv("/content/drive/MyDrive/sample_submission.csv")

In [5]:
training_data.head()

,Id,Runs,Boundaries,Six,Fifty,Hundred,Duck,Batting_Points,Wickets,4W_Haul,5W_Haul,Maidens,Bowling_Points,Total Points
0,1_Bipul Sharma,0,0,0,0,0,0,0,1,0,0,0,25,25
1,1_DJ Hooda,16,1,1,0,0,0,19,1,0,0,0,25,44
2,1_KM Jadhav,31,5,1,0,0,0,38,0,0,0,0,0,38
3,1_Mandeep Singh,24,5,0,0,0,0,29,0,0,0,0,0,29
4,1_Yuvraj Singh,62,10,3,1,0,0,86,0,0,0,0,0,86


**Data Cleaning**

In [6]:
training_data.describe()

,Runs,Boundaries,Six,Fifty,Hundred,Duck,Batting_Points,Wickets,4W_Haul,5W_Haul,Maidens,Bowling_Points,Total Points
count,15916.000000,15916.000000,15916.00000,15916.000000,15916.000000,15916.000000,15916.000000,15916.000000,15916.000000,15916.000000,15916.000000,15916.000000,15916.000000
mean,14.029027,1.794546,0.51332,0.070872,0.003895,0.057741,17.364036,0.501508,0.007791,0.001257,0.015142,12.741267,30.105303
std,19.932138,2.857668,1.14761,0.256619,0.062294,0.233260,26.402587,0.860593,0.087924,0.035427,0.123146,22.079906,30.143498
min,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,-2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-2.000000
25%,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000
50%,5.000000,0.000000,0.00000,0.000000,0.000000,0.000000,6.000000,0.000000,0.000000,0.000000,0.000000,0.000000,25.000000
75%,21.000000,3.000000,1.00000,0.000000,0.000000,0.000000,25.000000,1.000000,0.000000,0.000000,0.000000,25.000000,48.000000
max,175.000000,30.000000,17.00000,1.000000,1.000000,1.000000,255.000000,6.000000,1.000000,1.000000,2.000000,174.000000,305.000000


**No Missing Values present in the Training DataSet**

In [7]:
training_data.isna().sum()

Id                0
Runs              0
Boundaries        0
Six               0
Fifty             0
Hundred           0
Duck              0
Batting_Points    0
Wickets           0
4W_Haul           0
5W_Haul           0
Maidens           0
Bowling_Points    0
Total Points      0
dtype: int64

**Dropping the columns that we do not need and adding the Date Column as the Match ID column is not in Chronological order**

In [8]:
training_data["match_id"] = training_data["Id"].apply(lambda x: int(x.split("_")[0]))
training_data["player"] = training_data["Id"].apply(lambda x: x.split("_")[1])
training_data_date["date"] = pd.to_datetime(training_data_date["date"])
training_data.drop(training_data.iloc[:,1:13],axis=1,inplace=True)
training_data = training_data.merge(training_data_date.iloc[:,[0,3]],on = "match_id", how = "inner")
training_data_new = training_data.groupby(["player","date"],axis = 0, as_index= False).sum()
training_data_new

,player,date,Total Points,match_id
0,A Ashish Reddy,2012-01-05,25,348
1,A Ashish Reddy,2012-04-05,28,352
2,A Ashish Reddy,2012-04-26,50,341
3,A Ashish Reddy,2012-04-29,38,346
4,A Ashish Reddy,2012-05-18,12,373
...,...,...,...,...
15911,Z Khan,2017-06-05,2,44
15912,Z Khan,2017-08-04,51,5
15913,Z Khan,2017-10-05,0,49
15914,Z Khan,2017-11-04,75,9


**Importing the required Libraries.**

In [18]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn import metrics
from sklearn.ensemble import RandomForestRegressor
import xgboost
from sklearn.svm import SVR
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import Lasso

**Picking a single player to predict what will be their last 20 matches performance will be based on their previous matches. For that we going to make supervised data for the model. We know that players form can be known from last couple of matches. Here, I take last 3 matches played by Suresh Raina as the features and 4th match as the response variable.**

In [19]:
name = ["SK Raina"]
player_wise_data = pd.DataFrame()
player_wise_data["player"] = name
player_wise_data = player_wise_data.merge(training_data_new,on = "player", how = "inner")
#if player_wise_data.count()[0] >= 15:
player_wise_data_features = pd.DataFrame()
player_wise_data_features["feature_1"] = player_wise_data["Total Points"].shift(3)
player_wise_data_features["feature_2"] = player_wise_data["Total Points"].shift(2)
player_wise_data_features["feature_3"] = player_wise_data["Total Points"].shift(1)
player_wise_data_features["response"] =  player_wise_data["Total Points"]
player_wise_data_features = player_wise_data_features.dropna()
player_wise_data_features = player_wise_data_features.reset_index(drop= True)
player_wise_data_features

,feature_1,feature_2,feature_3,response
0,50.0,3.0,29.0,43
1,3.0,29.0,43.0,73
2,29.0,43.0,73.0,35
3,43.0,73.0,35.0,1
4,73.0,35.0,1.0,23
...,...,...,...,...
181,2.0,72.0,19.0,7
182,72.0,19.0,7.0,19
183,19.0,7.0,19.0,11
184,7.0,19.0,11.0,5


In [33]:
def get_stacking():
    # define the base models
    level0 = list()
    level0.append(('XGB', xgboost.XGBRegressor(verbosity=0)))
    level0.append(('RF', RandomForestRegressor()))
    level0.append(('svr', SVR()))
    level0.append(('lasso', Lasso(alpha=0.05)))
    # define meta learner model
    level1 = LinearRegression()
    # define the stacking ensemble
    s_model = StackingRegressor(estimators=level0, final_estimator=level1, cv=5)
    return s_model

**Starting with Linear Regression**

In [21]:
model1 = LinearRegression().fit(player_wise_data_features.iloc[0:165,0:3],player_wise_data_features.iloc[0:165,3])
y_hat1 = model1.predict(player_wise_data_features.iloc[165:,0:3]).astype(int)
print('RMSE:',np.sqrt(metrics.mean_squared_error(player_wise_data_features.iloc[165:,3],y_hat1)))

RMSE: 28.650604581005666


**Random Forest**

In [25]:
model2 = RandomForestRegressor().fit(player_wise_data_features.iloc[0:165,0:3],player_wise_data_features.iloc[0:165,3])
y_hat2 = model2.predict(player_wise_data_features.iloc[165:,0:3]).astype(int)
print('RMSE:',np.sqrt(metrics.mean_squared_error(player_wise_data_features.iloc[165:,3],y_hat2)))

RMSE: 30.422422808248022


**XGBoost**

In [29]:
model3 = xgboost.XGBRegressor(verbosity=0).fit(player_wise_data_features.iloc[0:165,0:3],player_wise_data_features.iloc[0:165,3])
y_hat3 = model3.predict(player_wise_data_features.iloc[165:,0:3]).astype(int)
print('RMSE:',np.sqrt(metrics.mean_squared_error(player_wise_data_features.iloc[165:,3],y_hat3)))

RMSE: 32.282237486625014


**Support Vector Regressor**

In [31]:
model4 = SVR().fit(player_wise_data_features.iloc[0:165,0:3],player_wise_data_features.iloc[0:165,3])
y_hat4 = model4.predict(player_wise_data_features.iloc[165:,0:3]).astype(int)
print('RMSE:',np.sqrt(metrics.mean_squared_error(player_wise_data_features.iloc[165:,3],y_hat4)))

RMSE: 26.25878537792275


**Stacking Ensemble**

In [34]:
model5 = get_stacking().fit(player_wise_data_features.iloc[0:165,0:3],player_wise_data_features.iloc[0:165,3])
y_hat5 = model5.predict(player_wise_data_features.iloc[165:,0:3]).astype(int)
print('RMSE:',np.sqrt(metrics.mean_squared_error(player_wise_data_features.iloc[165:,3],y_hat5)))

RMSE: 29.897443751216304


**Lasso**

In [35]:
model6 = Lasso(alpha=0.05).fit(player_wise_data_features.iloc[0:165,0:3],player_wise_data_features.iloc[0:165,3])
y_hat6 = model6.predict(player_wise_data_features.iloc[165:,0:3]).astype(int) 
print('RMSE:',np.sqrt(metrics.mean_squared_error(player_wise_data_features.iloc[165:,3],y_hat6)))

RMSE: 28.650604581005666


**After performing well with single player now I will now predict the last match played by every player. As there is no data for the new players, I would simply take the average of the last few matches played by the new players and assigning 0 to the debutants.**

In [40]:
def future_values(name,model_num):
    name = [name]
    player_wise_data = pd.DataFrame()
    player_wise_data["player"] = name
    player_wise_data = player_wise_data.merge(training_data_new,on = "player", how = "inner")
    if player_wise_data.count()[0] > 15:
        player_wise_data_features = pd.DataFrame()
        player_wise_data_features["feature_1"] = player_wise_data["Total Points"].shift(3)
        player_wise_data_features["feature_2"] = player_wise_data["Total Points"].shift(2)
        player_wise_data_features["feature_3"] = player_wise_data["Total Points"].shift(1)
        player_wise_data_features["response"] =  player_wise_data["Total Points"]
        player_wise_data_features = player_wise_data_features.dropna()
        player_wise_data_features = player_wise_data_features.reset_index(drop= True)
        if model_num == 1:
            model = LinearRegression().fit(player_wise_data_features.iloc[0:player_wise_data_features.shape[0]-1,0:3],player_wise_data_features.iloc[0:player_wise_data_features.shape[0]-1,3])
        elif model_num == 2:
            model = RandomForestRegressor().fit(player_wise_data_features.iloc[0:player_wise_data_features.shape[0]-1,0:3],player_wise_data_features.iloc[0:player_wise_data_features.shape[0]-1,3])
        elif model_num == 3:
            model = SVR().fit(player_wise_data_features.iloc[0:player_wise_data_features.shape[0]-1,0:3],player_wise_data_features.iloc[0:player_wise_data_features.shape[0]-1,3])
        elif model_num == 4:
            model = xgboost.XGBRegressor(verbosity=0).fit(player_wise_data_features.iloc[0:player_wise_data_features.shape[0]-1,0:3],player_wise_data_features.iloc[0:player_wise_data_features.shape[0]-1,3])
        elif model_num == 5:
            model = Lasso(alpha=0.05).fit(player_wise_data_features.iloc[0:player_wise_data_features.shape[0]-1,0:3],player_wise_data_features.iloc[0:player_wise_data_features.shape[0]-1,3])
        else:
            model = get_stacking().fit(player_wise_data_features.iloc[0:player_wise_data_features.shape[0]-1,0:3],player_wise_data_features.iloc[0:player_wise_data_features.shape[0]-1,3])
        y_hat = model.predict(player_wise_data_features.iloc[player_wise_data_features.shape[0]-1:,0:3]).astype(int)
    else:
        y_hat = player_wise_data["Total Points"].mean()
    if y_hat < -2:
        y_hat = -2
    return y_hat

In [41]:
def model_selection(model_num):
    response_fun = pd.DataFrame(columns = ["y_pred","y_test"])
    response_fun["player"] = training_data_new["player"].value_counts().index
    response_fun.set_index("player",inplace = True)
    for name in response_fun.index:
        response_fun.loc[name].y_pred = int(future_values(name,model_num))
        response_fun.loc[name].y_test = int(training_data_new["Total Points"][training_data_new["player"] == name].iloc[-1:])
    return response_fun    

# Comparing predicted data and test data

**We could perform the analysis for the players who have played more than 15 matches as new players doesn't have a lot of data.
If we use all data (including new players) then RMSE will be less as there are more than 50 players with only playing single match**

In [42]:
#Using Support Vector Regressor
response = model_selection(model_num=3)
print("RMSE using Support Vector Regressor : ",np.sqrt(metrics.mean_squared_error(response.y_pred[0:253],response.y_test[0:253])))

RMSE using Support Vector Regressor :  25.840572320782925


In [43]:
#Using XGBoost
response = model_selection(model_num=4)
print("RMSE using XGBoost : ",np.sqrt(metrics.mean_squared_error(response.y_pred[0:253],response.y_test[0:253])))

RMSE using XGBoost :  30.729085242517346


In [44]:
#Using RandomForest
response = model_selection(model_num=2)
print("RMSE using RandomForest : ",np.sqrt(metrics.mean_squared_error(response.y_pred[0:253],response.y_test[0:253])))

RMSE using RandomForest :  28.34959620070598


In [45]:
#Using Linear Regression
response = model_selection(model_num=1)
print("RMSE using Linear Regression : ",np.sqrt(metrics.mean_squared_error(response.y_pred[0:253],response.y_test[0:253])))

RMSE using Linear Regression :  26.350405566476326


In [46]:
#Using Lasso
response = model_selection(model_num=5)
print("RMSE using Lasso : ",np.sqrt(metrics.mean_squared_error(response.y_pred[0:253],response.y_test[0:253])))

RMSE using Lasso :  26.35370536682204


In [47]:
#Using Stacking
response = model_selection(model_num=6)
print("RMSE using Stacking : ",np.sqrt(metrics.mean_squared_error(response.y_pred[0:253],response.y_test[0:253])))

RMSE using Stacking :  28.069647991022634


# Predicting IPL 2020 player performance

**Removing Duplicate KD Karthik and Spliting Match Id and Player Name and adding Date column for sorting in chronological order.**

In [ ]:
submission = submission.drop(index=551).reset_index(drop= True)
submission["match_id"] = submission["Id"].apply(lambda x: int(x.split("_")[0]))
submission["player"] = submission["Id"].apply(lambda x: x.split("_")[1])
current_data_date["date"] = pd.to_datetime(current_data_date["date"])
submission = submission.merge(current_data_date.iloc[:,[0,2]],on = "match_id", how = "left")
submission_new = submission.groupby(["player","date"],axis = 0, as_index= False).sum()

**Removing NA values**

In [50]:
submission_new = submission_new.iloc[0:1279,:]
submission_new

,player,date,Total Points,match_id
0,A Mishra,2020-09-25,0,1216539
1,A Mishra,2020-09-29,0,1216532
2,A Mishra,2020-10-03,0,1216515
3,A Zampa,2020-09-28,0,1216547
4,A Zampa,2020-10-03,0,1216514
...,...,...,...,...
1274,YS Chahal,2020-10-25,0,1216544
1275,YS Chahal,2020-10-28,0,1216499
1276,YS Chahal,2020-10-31,0,1216502
1277,YS Chahal,2020-11-02,0,1216505


**Predicting Players performance match by match and adding the latest data in the training data for the next prediction.**

In [51]:
for i in range(0,submission_new.shape[0]):
    name = submission_new["player"][i]
    date = submission_new["date"][i]
    training_data_new.loc[training_data_new.shape[0]] = [name, date, 0, 1111]
    submission_new.loc[i,"Total Points"] = int(future_values(name,model_num=6))
    training_data_new.loc[training_data_new.shape[0]-1,"Total Points"] = submission_new["Total Points"][i]

**Result**

In [ ]:
submission_new

,player,date,Total Points,match_id
0,A Mishra,2020-09-25,28,1216539
1,A Mishra,2020-09-29,33,1216532
2,A Mishra,2020-10-03,25,1216515
3,A Zampa,2020-09-28,41,1216547
4,A Zampa,2020-10-03,41,1216514
...,...,...,...,...
1274,YS Chahal,2020-10-25,32,1216544
1275,YS Chahal,2020-10-28,30,1216499
1276,YS Chahal,2020-10-31,30,1216502
1277,YS Chahal,2020-11-02,30,1216505


**Final Submission Data has 1279 rows instead of 1283 because of one duplicate and 3 nan values**

In [ ]:
submission_final = pd.DataFrame()
submission_final["Id"] = submission_new["match_id"].astype(str) + "_" + submission_new["player"]
submission_final["Total Points"] = submission_new["Total Points"]
submission_final.loc[1279] = ["1216517_KD Karthik",0]
submission_final.loc[1280] = ["1216493_nan",0]
submission_final.loc[1281] = ["1216512_nan",0]
submission_final.loc[1282] = ["1216547_nan",0]
submission_final.sort_values(by= ["Id"],inplace = True)
submission_final.reset_index(drop = True)

,Id,Total Points
0,1216492_AS Yadav,26
1,1216492_AT Rayudu,31
2,1216492_DL Chahar,19
3,1216492_F du Plessis,36
4,1216492_HH Pandya,30
...,...,...
1278,1237181_S Hetmyer,39
1279,1237181_SA Yadav,16
1280,1237181_SS Iyer,30
1281,1237181_SS Tiwary,28


In [ ]:
submission_final.to_csv("submission.csv",index=False)